# Importing modules

In [1]:
__author__ = 'Pratyush'
import json
import numpy as np
import pandas as pd
from openpyxl import Workbook

import requests
import json
import os
import modules.config as config
from time import sleep
from requests.auth import HTTPBasicAuth
from ibm_watson import DiscoveryV1
from pprint import pprint
import modules.discovery_helper as dh_help
import modules.common_helper as cm_help
import sys
import pandas as pd
import modules.config as config
import math
import re
import hashlib
import unicodedata
from bs4 import BeautifulSoup 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [58]:
all_params = cm_help.get_params()

In [59]:
def discovery_init(COLLECTION_NAME=config.DISCOVERY_COLLECTION_NAME,default=all_params['discovery']['default']):
    response = {}
    
    discovery_param = all_params['discovery']
    # default = 
    authenticator = IAMAuthenticator(apikey=discovery_param[default]['apikey'])
    endurl = "https://gateway.watsonplatform.net/discovery/api"
    discovery = DiscoveryV1(
        version=discovery_param[default]['version'],
        authenticator = authenticator)
    iam_apikey=discovery_param[default]['apikey'],
    url=discovery_param[default]['url']    
    if default =="r3":
        discovery.set_service_url(discovery_param[default]['url'])

    environments = discovery.list_environments().get_result()
    # print(json.dumps(environments, indent=2))
    print("Discovery instance loaded "+default)
    js = json.dumps(environments)
    jres = json.loads(js)
    # print(jres['environments'][1]['environment_id'])
    env_id = jres['environments'][1]['environment_id']
    collections = discovery.list_collections(env_id).get_result()
    cols = json.dumps(collections, indent=2)
    colres = json.loads(cols)

    #print(colres['collections'])
    for item in colres['collections']:
        if item['name']== COLLECTION_NAME:
            print('COLLECTION ID:', item['collection_id'], 'COLLECTION NAME:', item['name'])
            col_id = item['collection_id']
    response['env_id'] = env_id
    response['col_id'] = col_id
    response['discovery'] = discovery
    return response

In [60]:
discovery_config = discovery_init(COLLECTION_NAME='Naviga1',default="r3")

Discovery instance loaded r3
COLLECTION ID: ec586f27-f848-40b7-99af-dab80a43b248 COLLECTION NAME: Naviga1


In [2]:
data_disc = json.load(open("first_50_sayan.json"))

In [3]:
data_spac = json.load(open("JSON_Spacy.json"))
data_spac = data_spac["All_sp"][:50]

In [4]:
data_html = json.load(open("first50.json"))

In [5]:
data_html

[{'id': '167424f8-ca7f-4224-a00b-a54a5a7766c7',
  'person': ['Kobe Bryant',
   'LeBron James',
   'Michael Jordan',
   'Bill Russel',
   'Robert Horry',
   'Scottie Pippen',
   'Magic Johnson',
   'Kareem Abdul-Jabbar',
   'Oscar Robertson',
   'Russel Westbrook',
   'Hakeem Olajuwon',
   'Larry Bird',
   'Larry Legend',
   "Shaquille O'Neal",
   'Ervin "Magic" Johnson',
   'Wilt Chamberlain',
   "Wilt Chamberlain's",
   'LeBon James',
   'Kareem Abdul Jabar',
   'John Wooden',
   'Lew Alcindor',
   'Babe Ruth'],
  'org': ["Chicago Bulls'",
   "ESPN's",
   'Boston Celtics',
   'Houston Rockets',
   'Orlando Magic',
   'Miami Heat',
   'UCLA',
   'Chicago Bulls',
   'Washington Wizards'],
  'location': ['Mount Rushmore', 'Los Angeles']},
 {'id': '7185ebf3-fa27-4fdc-a9fd-2011165a8603',
  'location': ['Des Plaines American Legion Post 36',
   'Edward Hotel',
   'Mannheim Road',
   'Rosemont',
   'Park Ridge',
   'Chicago'],
  'chron': ['Aug. 1', 'March 21, 2021', 'Friday, May 29'],
  'per

In [6]:
len(data_html)

50

# Creating sheet for ground truth

In [61]:
def add_entity_type(data,doc_no):
    for i in list(data[doc_no].keys())[1:]:
        for j in data[doc_no][i]:
            a.add(j)

In [62]:
def row_creator(data,doc_num,entity):
    global Discovery
    for i in list(data[doc_num].keys())[1:]:
        if entity in data[doc_num][i]:
            if len(Discovery)==0:
                Discovery+=(i)
            else:
                Discovery+=("/"+i)

In [63]:
df_doc_final = pd.DataFrame(columns=["Entity","Discovery","Spacy","Html","Doc_id","Text"])

In [64]:
document_id = ['167424f8-ca7f-4224-a00b-a54a5a7766c7',
 '7185ebf3-fa27-4fdc-a9fd-2011165a8603',
 '16601478-ebd3-4b55-bd54-e292865696e1',
 'a6f63029-6c69-49ea-a0dc-a5c7e85f55fb',
 '70ebe48c-a75d-4236-9ed6-4af0d8e1a421',
 '172ac60b-3611-4684-9c75-612abe362a84',
 '92123e9a-a03a-48f1-9f57-59319648b9a0',
 'a1433e93-fa9b-4f17-baaa-c64750ef109d',
 'e1410f9c-99a1-473c-83c3-fccb3b3b2ab2',
 'af8903b0-7658-4461-a770-960dbab328f0',
 'ca53af4f-991f-4712-81e0-ddd1591b1705',
 '79415524-ac65-4cee-99b7-18fc7265a0be',
 'ddb781fd-762a-4a61-9969-cac261425f63',
 'bda70352-5984-46a9-a891-119037c9c63d',
 '2ae29870-e433-4757-a94d-c495690238f5',
 'ff516124-45e4-460d-b6d9-48fad5f25c62',
 '2f872a57-25f2-45ea-ab4c-f02bf10c904c',
 '5eceab7f-f37e-44f4-aba1-19ad9b10f8da',
 'd5d89205-adb4-4139-8bf9-e958674a0da7',
 '0d06446d-a8fb-422d-a8a2-8e69c9be04ff',
 'f63cea4c-f478-4874-a350-41ec049ba56a',
 '58ca5429-675c-442c-98de-ad84fd2cf8c2',
 '60405989-66ba-421d-a062-5848eb7b6dc7',
 '97c847ae-5055-41ca-9f86-ea055963f1e7',
 'c637a047-60ff-4cb3-969e-5a0028b45cea',
 'bd8ad3d7-68ca-4d46-a82b-e1ea096afa26',
 'ddc9b34b-3d42-4153-b618-8d0da636c5c6',
 '6cda90e4-92dc-403b-9515-904ce543b138',
 '5723d949-29e0-4215-ba56-16f64ed33829',
 'e11c8456-3a7b-48c8-909f-1a5993b17091',
 'e3ea4a1c-e43f-4c4b-b491-87bf48adf89f',
 '1e948691-372e-4860-86e9-9f594ae98918',
 '11bab4cb-310f-43b0-8e0c-51dee6d2aa9f',
 '41e96803-31a4-47bf-9e5b-9af18497f0fb',
 '29733af7-66b6-461c-ac93-4cd20ef38ca8',
 'f6f391ca-749a-4bfa-9d9c-193104a5d237',
 'dcb1a853-c0a1-49a8-b2dd-674e06b80a39',
 'd5f19b61-ce53-4609-b802-857614b34537',
 'ff0587d0-5c06-412e-885a-31faf0151e77',
 '9717f3ec-6d4b-434e-82de-8a52552f8931',
 '9cb2e37a-1a40-4928-be23-d74ae391b89f',
 '87ae9d52-43a2-4b7c-9286-8acc7e2dd1fe',
 '0819535b-e9ea-4d53-9100-22435397bef7',
 '45404433-b299-4c0f-a938-08e88b550135',
 '9020f584-79c9-45c5-b8e4-d02b18227c2f',
 'c77ade55-521b-410e-8912-0ec5d0dfaf32',
 '9cc9f69d-ad0d-47ab-a0ef-79fb5584b301',
 '4b803714-d4b3-4420-baec-5b5334f2121d',
 'fb936514-3376-4f45-aabe-1688b1fd569f',
 '21b0f146-d8ea-4a82-9c76-8ffa13513b41']

In [68]:
for doc_id in range(50):
    a=set()
    for j in [data_disc,data_spac,data_html]:
        add_entity_type(j,doc_id)

    a=list(a)
    df_comp=pd.DataFrame(a,columns=["Entity"])
    
    df_row=pd.DataFrame(columns=["Discovery","Spacy","Html","Doc_id","Text"])

    offset = 0
    count = 1000
    reqparams = {'offset':offset,'count':count}
    reqparams['filters']='id::'+document_id[doc_id]
    discovery_data2 = dh_help.getDiscoveryData(discovery_config,reqparams)
    source = discovery_data2["results"][0]['text']
    
    for k in range(len(a)):
        Discovery=""
        row=[]
        for d in [data_disc,data_spac,data_html]:
            row_creator(d,doc_id,a[k])
            row.append(Discovery)
            Discovery=""           
        doc_id_ = document_id[doc_id]
        row.append(doc_id_)
        text = source
        row.append(text)
        df_row.loc[k]=row 
    df_doc=pd.concat([df_comp,df_row],axis=1) #horizontally for 1 id
    df_doc_final = df_doc_final.append(df_doc) #vertically for all ids

offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0


In [69]:
df_doc_final.shape

(2848, 6)

In [70]:
df_doc_final.head()

,Entity,Discovery,Spacy,Html,Doc_id,Text
0,138,,CARDINAL,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
1,two,,CARDINAL,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
2,NBA Finals,,ORG,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
3,The Black Mamba,,PERSON,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
4,second,,ORDINAL,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...


In [71]:
df_doc_final.to_excel("Ground_truth_V1.xlsx",index=False)

# Importing after GT after votes

In [15]:
data = pd.read_excel('Ground_truth_V2.xlsx')
data = pd.DataFrame(data)
data = data.fillna("")

In [73]:
data.head()

,Entity,ground truth,Discovery,Disc_vote,P_Discovery,J_Discovery,S_Discovery,Spacy,Spac_vote,P_Spacy,J_Spacy,S_Spacy,Html,Html_vote,P_Html,J_Html,S_Html
0,an eight year span,,,,,,,DATE,,,,,,,,,
1,NBA Finals,,,,,,,ORG,,,,,,,,,
2,third,,,,,,,ORDINAL,TP,Y,Y,Y,,,,,
3,All-NBA,,,,,,,PRODUCT,,,,,,,,,
4,ESPN's,,,,,,,,,,,,org,TP,Y,Y,Y


In [16]:
data.columns

Index(['Entity', 'ground truth', 'Discovery', 'Discovery_vote', 'P_Discovery',
       'J_Discovery', 'S_Discovery', 'Spacy', 'Spacy_vote', 'P_Spacy',
       'J_Spacy', 'S_Spacy', 'Html', 'Html_vote', 'P_Html', 'J_Html',
       'S_Html'],
      dtype='object')

In [17]:
data2 = pd.read_excel('Ground_truth_V1.xlsx')
data2 = pd.DataFrame(data2)
data2 = data2.fillna("")

In [18]:
ent_data = pd.DataFrame(columns = ['Entity', 'ground truth', 'Discovery', 'Discovery_vote', 'P_Discovery',
       'J_Discovery', 'S_Discovery', 'Spacy', 'Spacy_vote', 'P_Spacy',
       'J_Spacy', 'S_Spacy', 'Html', 'Html_vote', 'P_Html', 'J_Html',
       'S_Html'])

In [20]:
for i in range(len(data2)):
    for j in range(len(data)):
        if data2.loc[i,"Entity"] == data.loc[j,"Entity"]:
            row = data.loc[j]
            ent_data.loc[i] = row

In [21]:
ent_data.head()

,Entity,ground truth,Discovery,Discovery_vote,P_Discovery,J_Discovery,S_Discovery,Spacy,Spacy_vote,P_Spacy,J_Spacy,S_Spacy,Html,Html_vote,P_Html,J_Html,S_Html
0,138,,,,,,,CARDINAL,,Y,Y,Y,,,,,
1,two,,,,,,,CARDINAL,,Y,Y,Y,,,,,
2,NBA Finals,,,,,,,ORG,,,,,,,,,
3,The Black Mamba,,,,,,,PERSON,,,Y,,,,,,
4,second,,,,,,,ORDINAL,,Y,Y,,,,,,


# Generating ground truth

In [22]:
for i in range(len(ent_data)):
    count_Y = 0
    for k in ["Discovery","Spacy","Html"]:
        if (ent_data.loc[i,k] != ""):
            for j in ["P_"+k,"J_"+k,"S_"+k]:
                if (ent_data.loc[i,j] == "Y"):
                    count_Y+=1
            if (count_Y >=2):
                if (len(ent_data.loc[i,"ground truth"])==0):
                    ent_data.loc[i,"ground truth"] += ent_data.loc[i,k]
                else:
                    ent_data.loc[i,"ground truth"] += "/"+ent_data.loc[i,k]
        count_Y = 0

In [23]:
ent_data.head()

,Entity,ground truth,Discovery,Discovery_vote,P_Discovery,J_Discovery,S_Discovery,Spacy,Spacy_vote,P_Spacy,J_Spacy,S_Spacy,Html,Html_vote,P_Html,J_Html,S_Html
0,138,CARDINAL,,,,,,CARDINAL,,Y,Y,Y,,,,,
1,two,CARDINAL,,,,,,CARDINAL,,Y,Y,Y,,,,,
2,NBA Finals,,,,,,,ORG,,,,,,,,,
3,The Black Mamba,,,,,,,PERSON,,,Y,,,,,,
4,second,ORDINAL,,,,,,ORDINAL,,Y,Y,,,,,,


In [76]:
ent_data.to_excel("Ground_truth_V3.xlsx",index=False)

In [24]:
data2 = pd.read_excel('Ground_truth_V1.xlsx')
data2 = pd.DataFrame(data2)
data2 = data2.fillna("")

In [25]:
data2["Ground Truth"] = ent_data["ground truth"]

In [26]:
data2.head()

,Entity,Ground Truth,Discovery,Spacy,Html,Doc_id,Text
0,138,CARDINAL,,CARDINAL,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
1,two,CARDINAL,,CARDINAL,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
2,NBA Finals,,,ORG,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
3,The Black Mamba,,,PERSON,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
4,second,ORDINAL,,ORDINAL,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...


In [27]:
data2 = data2[['Entity', 'Ground Truth', 'Discovery', 'Spacy', 'Html','Doc_id','Text']]

In [28]:
data2.head()

,Entity,Ground Truth,Discovery,Spacy,Html,Doc_id,Text
0,138,CARDINAL,,CARDINAL,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
1,two,CARDINAL,,CARDINAL,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
2,NBA Finals,,,ORG,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
3,The Black Mamba,,,PERSON,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...
4,second,ORDINAL,,ORDINAL,,167424f8-ca7f-4224-a00b-a54a5a7766c7,2020 has been bizarre - you don't need me to t...


In [29]:
data2.to_excel("Ground_truth_V1.xlsx",index=False)

# Collecting TP, FP, TN, FN 

In [28]:
data2 = pd.read_excel('Ground_truth_V3.xlsx')
data2 = pd.DataFrame(data2)
data2["ground truth"].count()

2125

In [29]:
Total_GT = 2125

In [10]:
data2 = pd.read_excel('Ground_truth_V3.xlsx')
data2 = pd.DataFrame(data2)
data2 = data2.fillna("")

In [11]:
data2

,Entity,ground truth,Discovery,Discovery_vote,P_Discovery,J_Discovery,S_Discovery,Spacy,Spacy_vote,P_Spacy,J_Spacy,S_Spacy,Html,Html_vote,P_Html,J_Html,S_Html
0,an eight year span,,,,,,,DATE,,,,,,,,,
1,NBA Finals,,,,,,,ORG,,,,,,,,,
2,third,ORDINAL,,,,,,ORDINAL,TP,Y,Y,Y,,,,,
3,All-NBA,,,,,,,PRODUCT,,,,,,,,,
4,ESPN's,org,,,,,,,,,,,org,TP,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843,Va.,,Location,,,Y,,GPE,,,Y,,,,,,
2844,Enon,LOC/location,,,,,,LOC,,Y,Y,,location,,Y,Y,
2845,8News,,,,,,,CARDINAL,,,,,,,,,
2846,Wall,,Person,,,Y,,PERSON,,,Y,,,,,,


In [12]:
for i in range(len(data2)):
    
    lists = data2.loc[i,"ground truth"].split("/")
    
    for k in ["Discovery","Spacy","Html"]:
        
#         if(len(data2.loc[i,"ground truth"])==0 and len(data2.loc[i,k])!=0):
#             data2.loc[i,k+"_vote"] = "TN"
        
#         if(len(data2.loc[i,"ground truth"])!=0 and len(data2.loc[i,k])==0):
#             data2.loc[i,k+"_vote"] = "FN"

            
        if (data2.loc[i,k] in lists and len(data2.loc[i,k])!=0 and len(data2.loc[i,"ground truth"])!=0):
            data2.loc[i,k+"_vote"] = "TP"
        
        if (data2.loc[i,k] not in lists and len(data2.loc[i,k])!=0 and len(data2.loc[i,"ground truth"])!=0):
            data2.loc[i,k+"_vote"] = "FP"
        
    

In [14]:
data2

,Entity,ground truth,Discovery,Discovery_vote,P_Discovery,J_Discovery,S_Discovery,Spacy,Spacy_vote,P_Spacy,J_Spacy,S_Spacy,Html,Html_vote,P_Html,J_Html,S_Html
0,an eight year span,,,,,,,DATE,,,,,,,,,
1,NBA Finals,,,,,,,ORG,,,,,,,,,
2,third,ORDINAL,,,,,,ORDINAL,TP,Y,Y,Y,,,,,
3,All-NBA,,,,,,,PRODUCT,,,,,,,,,
4,ESPN's,org,,,,,,,,,,,org,TP,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843,Va.,,Location,,,Y,,GPE,,,Y,,,,,,
2844,Enon,LOC/location,,,,,,LOC,TP,Y,Y,,location,TP,Y,Y,
2845,8News,,,,,,,CARDINAL,,,,,,,,,
2846,Wall,,Person,,,Y,,PERSON,,,Y,,,,,,


In [15]:
data2.to_excel("Ground_truth_V4.xlsx",index=False)

# Precision & Recall

Precision = TP/(TP+FP)

Recall = TP/(TP+FN)

In simple terms, high precision means that an algorithm returned substantially more relevant results than irrelevant ones, while high recall means that an algorithm returned most of the relevant results.

In [16]:
data3 = pd.read_excel('Ground_truth_V4.xlsx')
data3 = pd.DataFrame(data3)
data3 = data3.fillna("")

In [17]:
data3

,Entity,ground truth,Discovery,Discovery_vote,P_Discovery,J_Discovery,S_Discovery,Spacy,Spacy_vote,P_Spacy,J_Spacy,S_Spacy,Html,Html_vote,P_Html,J_Html,S_Html
0,an eight year span,,,,,,,DATE,,,,,,,,,
1,NBA Finals,,,,,,,ORG,,,,,,,,,
2,third,ORDINAL,,,,,,ORDINAL,TP,Y,Y,Y,,,,,
3,All-NBA,,,,,,,PRODUCT,,,,,,,,,
4,ESPN's,org,,,,,,,,,,,org,TP,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843,Va.,,Location,,,Y,,GPE,,,Y,,,,,,
2844,Enon,LOC/location,,,,,,LOC,TP,Y,Y,,location,TP,Y,Y,
2845,8News,,,,,,,CARDINAL,,,,,,,,,
2846,Wall,,Person,,,Y,,PERSON,,,Y,,,,,,


## Discovery

In [18]:
data3["Discovery_vote"].value_counts()

      1960
TP     858
FP      30
Name: Discovery_vote, dtype: int64

In [19]:
Precision = 858/(858+30)
Recall = 858/Total_GT

Stats_disc = {"Precision": Precision,"Recall":Recall}

In [20]:
Stats_disc

{'Precision': 0.9662162162162162, 'Recall': 0.4037647058823529}

## Spacy

In [21]:
data3["Spacy_vote"].value_counts()

TP    1507
      1243
FP      98
Name: Spacy_vote, dtype: int64

In [22]:
Precision = 1507/(1507+98)
Recall = 1507/Total_GT

Stats_spac = {"Precision": Precision,"Recall":Recall}

In [23]:
Stats_spac

{'Precision': 0.9389408099688473, 'Recall': 0.7091764705882353}

## Html

In [24]:
data3["Html_vote"].value_counts()

      2035
TP     799
FP      14
Name: Html_vote, dtype: int64

In [25]:
Precision = 799/(799+14)
Recall = 799/Total_GT

Stats_Html = {"Precision": Precision,"Recall":Recall}

In [26]:
Stats_Html

{'Precision': 0.982779827798278, 'Recall': 0.376}

## Compare

In [27]:
print("--"*50)
print("Discovery")
print(Stats_disc)
print("--"*50)
print("Spacy")
print(Stats_spac)
print("--"*50)
print("HTML")
print(Stats_Html)
print("--"*50)

----------------------------------------------------------------------------------------------------
Discovery
{'Precision': 0.9662162162162162, 'Recall': 0.4037647058823529}
----------------------------------------------------------------------------------------------------
Spacy
{'Precision': 0.9389408099688473, 'Recall': 0.7091764705882353}
----------------------------------------------------------------------------------------------------
HTML
{'Precision': 0.982779827798278, 'Recall': 0.376}
----------------------------------------------------------------------------------------------------


## Discovery types

In [ ]:
entity_type=['Location',
 'Person',
 'Organization',
 'JobTitle',
 'Company',
 'Quantity',
 'Facility',
 'GeographicFeature',
 'EmailAddress',
 'PrintMedia',
 'HealthCondition',
 'Crime',
 'Sport',
 'Drug',
 'TwitterHandle',
 'Hashtag',
 'Broadcaster',
 'Movie',
 'Vehicle',
 'Award',
 'Anatomy',
 'NaturalEvent',
 'TelevisionShow',
 'SportingEvent',
 'MusicGroup',
 'IPAddress']

In [ ]:
counts = {"TN":0,"FN":0,"TP":0,"FP":0}
for i in entity_type:
    for j in range(len(data2)):
        lists = data2.loc[j,"ground truth"].split("/")
            if i in lists:
                counts[data.loc[j,"Discovery_vote"]]+=1
            
    

In [ ]:
entity_type=['Location',
 'Person',
 'Organization',
 'JobTitle',
 'Company',
 'Quantity']

In [ ]:
Ent_type_comp = {}
counts = {"TN":0,"FN":0,"TP":0,"FP":0}
for i in entity_type:
    for j in range(len(data)):
        lists = data.loc[j,"ground truth"].split("/")
        if i in lists or i == data.loc[j,"Discovery"]:
            counts[data.loc[j,"Discovery_vote"]]+=1
    precision = counts["TP"]/(counts["TP"]+counts["FP"])
    recall = counts["TP"]/(counts["TP"]+counts["FN"])
    
    b={i:{"Precision":precision,"Recall":recall}}
    
    Ent_type_comp.update(b)
    

In [ ]:
Ent_type_comp